In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("activities_dataset.csv")
df

,Activity ID,Activity Date,Activity Name,Activity Type,Activity Description,Elapsed Time,Distance,Max Heart Rate,Relative Effort,Commute,...,Intensity,Average Grade Adjusted Pace,Timer Time,Total Cycles,Recovery,With Pet,Competition,Long Run,For a Cause,Media
0,6655300337,"Feb 8, 2022, 11:40:14 AM",Night Run,Run,NaN,1694,5.04,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7130101042,"May 12, 2022, 12:24:20 PM",Morning Run,Run,NaN,1868,5.07,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7151864129,"May 16, 2022, 12:29:25 PM",Morning Run,Run,NaN,1778,5.06,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7168484206,"May 19, 2022, 12:26:25 PM",Morning Run,Run,NaN,2094,6.09,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7194771847,"May 24, 2022, 12:01:09 PM",Morning Run,Run,NaN,1449,4.25,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,16749275101,"Dec 15, 2025, 2:10:26 PM",Morning Weight Training,Workout,"Barbell Squats (155 lbs x 6, 160 lbs x 4)\nLeg...",2704,0.00,163.0,6.0,False,...,NaN,NaN,NaN,226.0,0.0,NaN,NaN,NaN,NaN,NaN
445,16758470948,"Dec 16, 2025, 2:09:45 PM",Morning Weight Training,Workout,"Incline Barbell Bench Press (135 lbs x 5, 135 ...",2149,0.00,138.0,4.0,False,...,NaN,NaN,NaN,106.0,0.0,NaN,NaN,NaN,NaN,NaN
446,16776946737,"Dec 18, 2025, 2:09:25 PM",Morning Weight Training,Workout,"Weighted Pullups (+25 lbs x 7, +25 lbs x 6)\nS...",2605,0.00,159.0,6.0,False,...,NaN,NaN,NaN,140.0,0.0,NaN,NaN,NaN,NaN,NaN
447,16795369450,"Dec 20, 2025, 4:31:59 PM",Morning Weight Training,Workout,"Barbell Squats (155 lbs x 6, 160 lbs x 4)\nEZ ...",2450,0.00,179.0,6.0,False,...,NaN,NaN,NaN,106.0,0.0,NaN,NaN,NaN,NaN,NaN
